# [o3]- Proyecto Ozono - ETL_Join - v0

    0. Inicialización 
    1. Datos
        1.0 Carga de ficheros (Aire, Clima, Estaciones y Calendario)
        1.1 Paso a Pandas
        1.2 Tipos
            1.2.0 Estaciones
            1.2.1 Aire
            1.2.2 Clima
            1.2.3 Calendario
        1.3 Darle grupo a estaciones de aire        
    2. Unir AIRE + CLIMA
        2.0 Separamos datos contaminacion (2014-18 | 2019-NOW)
        2.1 MERGE (ESTACION AIRE, DIA) + (DATOS CLIMA )
        2.2 Unimos partes generadas por el MERGE       
    3. Incluir CALENDARIO   
    4. Formato  
    5. Exportar
    EXTRA. CHECKEO


# [0] - Inicialización

In [1]:
import findspark
findspark.init('/home/rulicering/BigData/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,FloatType
import re as reg
import numpy as np
import datetime

In [2]:
spark = SparkSession.builder.appName('join').getOrCreate()

# [1] Datos

## [1.0] - Carga de ficheros (Aire, Clima, Estaciones y Calendario)

In [3]:
df_aire = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Contaminacion/Contaminacion_diaria_2014-NOW.csv',inferSchema= True,header=True)
df_clima = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Clima/Clima_diario_2014-NOW.csv',inferSchema= True,header=True)
df_estaciones = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Estaciones/Estaciones.csv',inferSchema= True,header=True)
df_calendario = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Calendario/Calendario_2013-2020.csv',inferSchema= True,header=True)

## [1.1] - Paso a Pandas

In [4]:
pd_aire = df_aire.drop("_c0").toPandas()
pd_clima = df_clima.drop("_c0").toPandas()
pd_estaciones = df_estaciones.drop("_c0").toPandas()
pd_calendario = df_calendario.drop("_c0").toPandas()

## [1.2] - Tipos
    
    CODIGO_CORTO -> STRING/OBJECT
    ANO -> INTEGER
    MES -> INTEGER
    DIA -> INTEGER
    FECHA -> INTEGER
    MEDICIONES -> DOUBLE

### [1.2.0] -  Estaciones

In [5]:
#None to NULO
pd_estaciones = pd_estaciones.replace(('None',None),np.nan)

In [6]:
#Columnas estaciones medicion - 2014
regex = reg.compile("E_AEMET")
c_grupo_14 = [elem for elem in list(filter(regex.search,df_estaciones.columns))]
c_grupo_14

['E_AEMET',
 'E_AEMET_HOY_89',
 'E_AEMET_HOY_87',
 'E_AEMET_HOY_82',
 'E_AEMET_HOY_81',
 'E_AEMET_HOY_83']

In [7]:
#Columnas estaciones medicion - 2019
regex = reg.compile("E_\d\d")
c_estacionxmagnitud_19 = [elem for elem in list(filter(regex.search,df_estaciones.columns))]
c_estacionxmagnitud_19

['E_81', 'E_82', 'E_83', 'E_87', 'E_89']

In [8]:
#Columnas estaciones medicion a string
for columna in c_estacionxmagnitud_19:
    pd_estaciones[columna] = pd_estaciones[columna].astype(str)

In [9]:
#pd_estaciones.dtypes

###  [1.2.1] - Aire

In [10]:
#None to NULO
pd_aire = pd_aire.replace(('None',None),np.nan)

In [11]:
#CODIGOS CORTOS A Strig
pd_aire["CODIGO_CORTO"] = pd_aire["CODIGO_CORTO"].astype(str)

In [12]:
pd_aire = pd_aire.sort_values(by="FECHA")

In [13]:
#pd_aire.dtypes

### [1.2.2] - Clima

In [14]:
#None to NULO
pd_clima = pd_clima.replace(('None',None),np.nan)

In [15]:
#Listar las columnas que miden magnitudes
columnas_valoresmagnitudes = list(pd_clima.columns)[5:]

In [16]:
#Columnas magnitudes a float
for columna in columnas_valoresmagnitudes:
    #Comas por puntos
    pd_clima[columna]  =  [reg.sub(',','.',str(x)) for x in pd_clima[columna]]
    # String to float
    pd_clima[columna] = pd_clima[columna].astype(float)

In [17]:
pd_clima = pd_clima.sort_values(by="FECHA")

In [18]:
#pd_clima.dtypes

### [1.2.3] - Calendario

In [19]:
pd_calendario = pd_calendario.replace(('None',None),np.nan)

## [1.3] - Darle grupo a estaciones de AIRE

    2014-2018 -> E_AEMET
    
    2019-NOW -> 81,82,83,87,89

In [20]:
#Sacamos estaciones de aire y codigos de estaciones de clima asociadas
pd_estaciones_aire = pd_estaciones[pd_estaciones["MIDE_AIRE"]>0]

In [21]:
#Nos quedamos con las columnas que queremos
c_agrupamiento = ["CODIGO_CORTO"] + c_grupo_14 + c_estacionxmagnitud_19
pd_estaciones_aire = pd_estaciones_aire[c_agrupamiento]

In [22]:
#Unimos ambos datasets
pd_aire = pd_aire.merge(pd_estaciones_aire, on =["CODIGO_CORTO"])

# [2] -  UNIR AIRE + CLIMA

## [2.0] - Separamos datos contaminacion (2014-18 | 2019-NOW)

In [23]:
#Datos 14-18
pd_datos_14_18 = pd_aire[pd_aire["FECHA"]<20190101]

In [24]:
#Datos 19-NOW
pd_datos_19_NOW = pd_aire[pd_aire["FECHA"]>=20190101]

## [2.1] - MERGE (ESTACION AIRE, DIA) + (DATOS CLIMA )

In [25]:
# Separamos columnas de info (comunes) y las columnas de datos
columnas = list(pd_clima.columns) 
c_info = columnas[:5]
c_magnitudes = columnas[5:]

In [26]:
#Asociamos cada columna de datos de clima (magnitud) a la estación de aire correspondiente.  
for magnitud in c_magnitudes:
    cols = c_info.copy()
    cols.append(magnitud)
    
    pd_clima_magnitud = pd_clima[cols]

    #2014-2018
    pd_clima_magnitud_14_18 = pd_clima_magnitud.rename(columns={"CODIGO_CORTO":c_grupo_14[0]})
    pd_datos_14_18 = pd_datos_14_18.merge(pd_clima_magnitud_14_18,on = ["ANO", "MES", "DIA","FECHA",c_grupo_14[0]])
    
    #2019-NOW
    pd_clima_magnitud_19_NOW = pd_clima_magnitud.rename(columns={"CODIGO_CORTO":"E_%s"%magnitud})
    pd_datos_19_NOW = pd_datos_19_NOW.merge(pd_clima_magnitud_19_NOW,on = ["ANO", "MES", "DIA","FECHA","E_%s"%magnitud])

## [2.2] - Unimos partes generadas por el MERGE

In [27]:
#pd_datos_14_18.dtypes

In [28]:
#pd_datos_19_NOW.dtypes

In [29]:
pd_datos = pd.concat([pd_datos_14_18,pd_datos_19_NOW])

# [3] - Incluir CALENDARIO

In [30]:
pd_datos_y_calendario = pd_datos.merge(pd_calendario, on = "FECHA")

# [4] - FORMATO

In [37]:
#Borramos las columnas utilizadas para relacionar estaciones de aire y clima
a_borrar = c_grupo_14 + c_estacionxmagnitud_19

In [38]:
pd_final = pd_datos_y_calendario.drop(columns = a_borrar)

In [40]:
cols = pd_final.columns.tolist()

In [42]:
cols = cols[0:5]+ cols[-3:]+ cols [5:-3]

In [44]:
pd_final = pd_final[cols]

# [5] - EXPORTAR

In [47]:
#Versiones
hoy = datetime.date.today().strftime("%Y-%m-%d")
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/BackUp/Datos_2014-NOW-" + hoy + ".csv")

In [48]:
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/Datos_2014-NOW.csv")

# [EXTRA] - CHECKEO

In [262]:
#pd_chequeo = pd_datos[['CODIGO_CORTO', 'FECHA','E_AEMET','E_81', 'E_82', 'E_83', 'E_87', 'E_89', '81', '82', '83', '87', '89']]

In [261]:
#2014-2018
#pd_chequeo[(pd_datos["FECHA"]==20140101)].sort_values(by="E_AEMET")
#pd_clima[pd_clima["FECHA"]==20140101]

#2019-NOW
#pd_chequeo[(pd_datos["FECHA"]==20190101)]
#pd_clima[pd_clima["FECHA"]==20190101]